# Import necessary libraries

In [1]:
import pandas as pd
import numpy as np
import scipy
from sklearn.cluster import KMeans
from sentence_transformers import SentenceTransformer

C:\Users\Savannah\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\Savannah\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\Savannah\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\Savannah\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:529: FutureWa

# Load model

In [2]:
model = SentenceTransformer('bert-base-nli-mean-tokens')

# Load tag data

In [3]:
column = "space" # Set to "nospace" or "space"
path = "../data/tags.xlsx"
transcriber_df = pd.read_excel(path, converters={'space':str, 'nospace':str}, sheet_name="ZooniverseTags")
expert_df = pd.read_excel(path, converters={'space':str, 'nospace':str}, sheet_name="ExpertTags")
expert_tags = expert_df["nospace"].tolist()

# Get tag embeddings

In [4]:
expert_embed = model.encode(expert_df[column].tolist())
transcriber_embed = model.encode(transcriber_df[column].tolist())

# Compare embeddings

In [7]:
def compare_all(i, tag, embed):
    distances = scipy.spatial.distance.cdist([embed], expert_embed, "cosine")[0]
    ranking = sorted(zip(range(len(distances)), distances), key=lambda x: x[1])
    return {
        "index": i,
        "transcriber_tag": tag,
        "expert_tag_1": expert_tags[ranking[0][0]],
        "score_1": 1-ranking[0][1],
        "expert_tag_2": expert_tags[ranking[1][0]],
        "score_2": 1-ranking[1][1],
        "expert_tag_3": expert_tags[ranking[2][0]],
        "score_3": 1-ranking[2][1]
    }
    
results = [compare_all(i, tag, embed) for i, tag, embed in 
           zip(range(len(transcriber_df)), transcriber_df['nospace'].tolist(), transcriber_embed)]

# Save as .csv

In [11]:
# Convert list of results to dataframe
df = pd.DataFrame(results)
# Sort results by cosine similarity
df = df.sort_values('score_1', ascending=False)
# Write to csv
df.to_csv("../data/tag_approx_"+column+".csv", index=False)

# Cluster tags lacking close match

In [13]:
k = 30 # Adjust number of clusters

cluster_results = [results[i] for i in range(len(results))
                   if results[i]['score_1'] < 0.65 ] # Adjust similarity threshold

cluster_embeds = [transcriber_embed[cluster_results[i]['index']] 
                  for i in range(len(cluster_results))]

kmeans = KMeans(n_clusters=k).fit(cluster_embeds)

for i in range(k):
    print("Cluster", i)
    print([cluster_results[j]['transcriber_tag'] 
           for j in range(len(kmeans.labels_))
           if kmeans.labels_[j] == i])

Cluster 0
['barbershops', 'busservice', 'cobbler', 'courtmarshal', 'legalopportunities', 'merchantmarines', 'phonebooth', 'salesgirls', 'shippingspace', 'taxationistheft', 'taxmoney', 'telephone']
Cluster 1
['agriculture', 'economist', 'farmboy', 'farmboys', 'farmer', 'farmers', 'farming', 'farmlife', 'forestry', 'forests', 'fruitfarm', 'lumberjack', 'ranches', 'servingthecountry', 'wewantseasonaluniforms']
Cluster 2
['30days', '30yearolds', '30years', '30yearsold', '30yrsplus', '3months', '4plusyears', '4yearsor35', 'doublehours', 'fouryearsexperience', 'prewarsalary', 'thirtyfive', 'threedaypass', 'twoyears']
Cluster 3
['brownnose', 'greendrafters', 'moremilk', 'white_supremacy', 'whiteofficers', 'whitesoldier', 'whitesoldiers', 'whitesupremacy']
Cluster 4
['court-martial', 'court-martials', 'officerversusenlisted', 'religionoverrace', 'too_old_to_fight', 'toooldforflying', 'worldwar2soldier']
Cluster 5
['beefstew', 'diabetes', 'food_poisoning', 'food-shortages', 'pigfood']
Cluster 6